In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
keras.__version__

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
fashion_mnist

In [6]:
(X_train_full, y_train_full),(X_test,y_test)=fashion_mnist.load_data()

In [7]:
X_train_full.shape

In [8]:
#Scaling the pixels to range between 0 and 1
X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [9]:
y_train

In [10]:
#Creating a Sequential model
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [11]:
#Alternate way to create Sequential model 
model=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [12]:
model.summary()

In [13]:
model.layers[1]

In [14]:
#weights and biases
weights,biases = model.layers[1].get_weights()      

In [15]:
weights

In [16]:
weights.shape          #28*28=784 and 300 - 1 for each neuron

In [17]:
biases.shape    #It containes 300 Biases terms - 1 for each neuron

In [18]:
#compiling MLP model
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [19]:
X_train.shape

In [20]:
#Fitting and evaluating the model
history=model.fit(X_train,y_train,epochs=30,validation_data=(X_valid,y_valid))

In [21]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.gca().set_xlim(0,30)
plt.show()

In [22]:
model.evaluate(X_test,y_test)

In [23]:
class_name=['T-shirt/Top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']

In [24]:
X_new=X_test[:3]
y_proba=model.predict(X_new)
y_proba.round(2)

In [25]:
#Building Regression MLP using Sequential API
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test=train_test_split(housing.data,housing.target)

In [26]:
X_train, X_valid, y_train, y_valid=train_test_split(X_train_full, y_train_full)

In [27]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)

In [28]:
reg_model=keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [29]:
#Compiling the model
reg_model.compile(loss='mean_squared_error',optimizer='sgd')
history=reg_model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

In [30]:
mse_test=reg_model.evaluate(X_test,y_test)

In [31]:
X_new=X_test[:3]
X_pred=reg_model.predict(X_new)

In [32]:
X_pred

In [62]:
#Building Complex Model Using Functional API
input_=keras.layers.Input(shape=X_train.shape[1:])
hidden1=keras.layers.Dense(30,activation='relu')(input_)
#hidden2=keras.layers.Dense(30,activation='relu')(hidden1)
concat=keras.layers.Concatenate()([input_,hidden1])
output=keras.layers.Dense(1)(concat)

func_model = keras.Model(inputs=[input_],outputs=[output])

In [63]:
func_model.summary()

In [64]:
func_model.compile(loss='mse',optimizer='sgd')

In [55]:
print(1e-3)

In [56]:
X_train[1:].shape

In [65]:
history=func_model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

In [67]:
#Creating 2inputs
input_A=keras.layers.Input(shape=[5],name='wide_input')
input_B=keras.layers.Input(shape=[6],name='deep_input')
hidden1=keras.layers.Dense(30,activation='relu')(input_B)
hidden2=keras.layers.Dense(30,activation='relu')(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1)(concat)

model_aux=keras.Model(inputs=[input_A,input_B],outputs=[output])

In [68]:
model_aux.compile(loss='mse',optimizer='sgd')

In [74]:
X_train_A, X_train_B = X_train[:,:5], X_train[:,2:]
X_valid_A, X_valid_B = X_valid[:,:5], X_valid[:,2:]
X_test_A, X_test_B = X_test[:,:5], X_test[:,2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [70]:
history=model_aux.fit((X_train_A,X_train_B),y_train,epochs=20,validation_data=((X_valid_A,X_valid_B),y_valid))

In [75]:
mse_test=model_aux.evaluate((X_test_A,X_test_B),y_test)

In [76]:
y_pred=model_aux.predict((X_new_A,X_new_B))

In [77]:
y_pred

In [79]:
#Creating 2inputs
input_A=keras.layers.Input(shape=[5],name='wide_input')
input_B=keras.layers.Input(shape=[6],name='deep_input')
hidden1=keras.layers.Dense(30,activation='relu')(input_B)
hidden2=keras.layers.Dense(30,activation='relu')(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name='main_output')(concat)
aux_output=keras.layers.Dense(1,name='aux_output')(hidden2)

model_aux=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])

In [81]:
model_aux.compile(loss=['mse','mse'],loss_weights=[0.9,0.1],optimizer='sgd')   #Assigning more weightage to the main output

In [83]:
history=model_aux.fit([X_train_A,X_train_B],[y_train,y_train],epochs=20,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]))

In [85]:
total_loss, main_loss, aux_loss = model_aux.evaluate([X_test_A,X_test_B],[y_test,y_test])

In [86]:
y_pred_main,y_pred_aux=model_aux.predict([X_new_A,X_new_B])

In [87]:
y_pred_main,y_pred_aux

In [91]:
#Callbacks
checkpoint_cb=keras.callbacks.ModelCheckpoint('my_keras_model.h5',save_best_only=True)
history=model_aux.fit([X_train_A,X_train_B],[y_train,y_train],epochs=20,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),callbacks=[checkpoint_cb])

In [92]:
early_stopping_cb=keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
history=model_aux.fit([X_train_A,X_train_B],[y_train,y_train],epochs=20,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),callbacks=[early_stopping_cb])

In [101]:
#Tuning Parameters
def build_model(n_hidden=1,n_neurons=30,learning_rate=3e-3,input_shape=[8]):
    model=keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer=keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse',optimizer=optimizer)
    return model

In [102]:
#Build a model using the above function
keras_reg=keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [103]:
keras_reg.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=early_stopping_cb)

In [104]:
mse_test=keras_reg.score(X_test,y_test)

In [ ]:
#Implementing RandomizedSearchCV
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist={
    'n_hidden':[0,1,2,3],
    'n_neurons':np.arange(1,100),
    'learning_rate':reciprocal(3e-4,3e-2)
}

rnd_search_cv=RandomizedSearchCV(keras_reg,param_dist,n_iter=10,cv=3)
rnd_search_cv.fit(X_train,y_train,epochs=50,validation_data=(X_valid,y_valid),callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
rnd